# Getting Full Papers

In [9]:
import urllib
import requests
import os
import math
import json
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np

from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import igemutils as igem

email = 'ivalexander13@berkeley.edu' # enter ur email pls

## Create Subsets

In [10]:
rxns_pmid = igem.get_json("./get_metadata_pmidset/unpaywall_metadata_pmids.json")
oa_subset, non_oa_subset = igem.generate_subset(rxns_pmid, 'is_oa', [True], return_complement=True)

Count of is_oa being [True]: 46843.
Count of is_oa not matching [True]: 49375.


# 1. Elsevier!!

## Splitting Data

In [11]:
n_splits = 6 # fill this in!!

elsevier_subset = igem.generate_subset(non_oa_subset, 'publisher', ['Elsevier', 'Elsevier BV'])[0]
igem.split_dict(elsevier_subset, n_splits, 'get_fullpapers/elsevier/elsevier_subset_split')

Count of publisher being ['Elsevier', 'Elsevier BV']: 23267.
Saved part 1.
Saved part 2.
Saved part 3.
Saved part 4.
Saved part 5.
Saved part 6.
All done!


## Run this in parallel!!

```python get_elsevier.py <num>```

Where <num> is the split# generated from the cell above.
    
For convenience: just paste these in (in separate terminals!)

In [12]:
for n in range(1, n_splits + 1):
    print(f"python get_elsevier.py {n}")
    print("")
    

python get_elsevier.py 1

python get_elsevier.py 2

python get_elsevier.py 3

python get_elsevier.py 4

python get_elsevier.py 5

python get_elsevier.py 6



# 2. Open Access

## Splitting Data

In [13]:
n_splits = 6 # fill this in!!
igem.split_dict(oa_subset, n_splits, 'get_fullpapers/oa/oa_subset_split')

Saved part 1.
Saved part 2.
Saved part 3.
Saved part 4.
Saved part 5.
Saved part 6.
All done!


In [14]:
oa_split1 = igem.get_json('get_fullpapers/oa/oa_subset_split1.json')

In [15]:
len(oa_split1)

7808

### Convert PDF to Text

In [16]:
def get_bestlink(pmid: str, oa_subset: dict):
    try:
        out = oa_subset[pmid]['best_oa_location']['url_for_pdf']
    except:
        return False
    return out

In [81]:
oa_split1['23277574']['best_oa_location']['url_for_pdf']

'https://www.pnas.org/content/pnas/110/3/954.full.pdf'

In [82]:
try:
    response = requests.get('https://www.pnas.org/content/pnas/110/3/954.full.pdf')    
except (requests.exceptions.RequestException,
        ConnectionResetError) as e:
    print("AAAAA")

In [83]:
with open('get_fullpapers/oa/tmp.pdf', 'wb') as fd:
    fd.write(response.content)

In [85]:
import subprocess
def pdf_to_text(filepath):
    print('Getting text content for {}...'.format(filepath))
    process = subprocess.Popen(['pdf2txt.py', filepath], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    stdout, stderr = process.communicate()

    if process.returncode != 0 or stderr:
        raise OSError('Executing the command for {} caused an error:\nCode: {}\nOutput: {}\nError: {}'.format(filepath, process.returncode, stdout, stderr))
    print('Done.')
    return stdout.decode('utf-8')

fulltext_str = pdf_to_text('get_fullpapers/oa/tmp.pdf')

Getting text content for get_fullpapers/oa/tmp.pdf...
Done.


In [86]:
fulltext_str.replace('\n', ' ')

'Mechanism of proton-coupled quinone reduction in Photosystem II  Keisuke Saitoa,b, A. William Rutherfordc, and Hiroshi Ishikitaa,b,1  aCareer-Path Promotion Unit for Young Life Scientists, Graduate School of Medicine, Kyoto University, Kyoto 606-8501, Japan; bJapan Science and Technology Agency, Precursory Research for Embryonic Science and Technology, Saitama 332-0012, Japan; and cMolecular Biosciences, Imperial College London, London SW7 2AZ, United Kingdom  Edited by Pierre A. Joliot, Institut de Biologie Physico-Chimique, Paris, France, and approved November 28, 2012 (received for review July 27, 2012)  Photosystem II uses light to drive water oxidation and plastoqui- none (PQ) reduction. PQ reduction involves two PQ cofactors, QA and QB, working in series. QA is a one-electron carrier, whereas QB undergoes sequential reduction and protonation to form QBH2. QBH2 exchanges with PQ from the pool in the membrane. Based on the atomic coordinates of the Photosystem II crystal structure

In [91]:
def get_paper(pmid: str, metadata: dict) -> str:
    url = metadata['best_oa_location']['url_for_pdf']
    out_pdf = f'get_fullpapers/oa/tmp{num_run}.pdf'
    
    try:
        # Fetch the paper
        response = requests.get(url)

        # Save to tmp.pdf
        with open(out_pdf, 'wb') as fd:
            fd.write(response.content)
            fd.close()

        outtext = pdf_to_text(out_pdf)
    except Exception:
        return False
    except (requests.exceptions.RequestException,
        ConnectionResetError):
        print("Connection Error")
        return False


    return outtext.replace('\n', ' ')

In [93]:
num_run = 1
get_paper('23277574', oa_split1['23277574'])

Getting text content for get_fullpapers/oa/tmp1.pdf...
Done.


'Mechanism of proton-coupled quinone reduction in Photosystem II  Keisuke Saitoa,b, A. William Rutherfordc, and Hiroshi Ishikitaa,b,1  aCareer-Path Promotion Unit for Young Life Scientists, Graduate School of Medicine, Kyoto University, Kyoto 606-8501, Japan; bJapan Science and Technology Agency, Precursory Research for Embryonic Science and Technology, Saitama 332-0012, Japan; and cMolecular Biosciences, Imperial College London, London SW7 2AZ, United Kingdom  Edited by Pierre A. Joliot, Institut de Biologie Physico-Chimique, Paris, France, and approved November 28, 2012 (received for review July 27, 2012)  Photosystem II uses light to drive water oxidation and plastoqui- none (PQ) reduction. PQ reduction involves two PQ cofactors, QA and QB, working in series. QA is a one-electron carrier, whereas QB undergoes sequential reduction and protonation to form QBH2. QBH2 exchanges with PQ from the pool in the membrane. Based on the atomic coordinates of the Photosystem II crystal structure

In [33]:
os.remove('tmp.pdf')

In [46]:
import PyPDF2
import io

outtext = ''
with io.BytesIO(response.content) as f:
    reader = PyPDF2.PdfFileReader(f)
    for page in reader.pages:
        outtext += page.extractText()

### Command for bash
Make sure the pwd is /brenda_data_collection/, then paste the following command:

In [80]:
for i in range(1, 7):
    print(f"python fetch_oa.py {i} &", end=' ')

python fetch_oa.py 1 & python fetch_oa.py 2 & python fetch_oa.py 3 & python fetch_oa.py 4 & python fetch_oa.py 5 & python fetch_oa.py 6 & 

***

## [1.] DO NOT RUN: Elsevier API and Loop
Code below is the rough basis for elsevier.py

Setting up the API

In [ ]:
# Load configuration
con_file = open("elsevier_api/config.json")
config = json.load(con_file)
con_file.close()

# Initialize client
client = ElsClient(config['apikey'])
client.local_dir = "./get_fullpapers_pmidset/"

In [ ]:
# Function: Get paper
def get_paper(doi: str):
    ## ScienceDirect (full-text) document example using DOI
    doi_doc = FullDoc(doi = doi)
    if doi_doc.read(client):
        return doi_doc.data
    else:
        return False

Main Loop (taken and modified from elsevier_api).

**NOTE: MUST ENABLE BERKELEY VPN FOR THIS TO WORK!**

In [ ]:
# Test to see if it works
get_paper('10.1016/0031-9422(73)80493-5');

In [ ]:
# Output file = doi to full_text dict dictionary (in json)
def loop_elsevier(elsevier_subset, outfile):
    json_file = outfile
    if os.path.isfile(json_file):
        with open(json_file, 'r') as fp:
            elsevier_fulltexts = json.load(fp)
            fp.close()
    else:
        elsevier_fulltexts = {}

    # hyperparam (-1 if max)
    max_calls = -1

    # Stats
    calls = 0
    fails = 0
    not_elsevier = 0
    successes_or_found = 0
    queries = 0

    try:
        # Looping through quantify_dataset output json.
        for pmid, metadata in elsevier_subset.items():
            doi = metadata['doi']
            doi_url = f"https://doi.org/{doi}"

            # dont go over max calls. (-1 if infinite)
            if calls == max_calls or (queries == -1):
                print("Query limit reached.")
                break
            else:
                calls += 1

            # checks if paper has been successfully fetched before
            if pmid in elsevier_fulltexts:
                successes_or_found += 1
                print(f"## Call {calls} found.")
                continue

            fullpaper = get_paper(doi)
            if (fullpaper):
                elsevier_fulltexts[pmid] = fullpaper
                successes_or_found += 1
                queries += 1
                print(f"Call {calls} success.")
            else:
                fails += 1
                print (f"##### Call {calls} failed: {fails}. DOI: {doi_url}.")

    except KeyboardInterrupt:
        pass

    # save to file
    with open(json_file, 'w') as fp:
        json.dump(elsevier_fulltexts, fp)
        # vary: alter frequency of file save
        fp.close()

    # Print Stats
    print("")
    print("###### STATS ######")
    print(f"Total calls: {calls}")
    print(f"Total number of queries: {queries}")
    print(f"Total number of Elsevier papers: {calls - not_elsevier}")
    print(f"Number of Non-Elsevier papers skipped: {not_elsevier}")
    print(f"Number of fetch failures: {fails}")
    print(f"Papers in storage: {len(elsevier_fulltexts)}")
    print(f"% of success: {successes_or_found / (calls-not_elsevier) * 100}%")